In [1]:
import pandas as pd
import time
from tqdm import tqdm
import numpy as np
import duckdb as db

In [2]:
conn = db.connect('../data/RE_moscow_db.duckdb')
print(conn.execute('PRAGMA show_tables').df())
avito = conn.execute('select * from user_ads').df()
df = conn.execute('''
                select 
                *

                from home_info
                ''').df()
conn.close()

          name
0   agency_ads
1  builder_ads
2    home_info
3     user_ads


In [33]:
avito['clean_adress'] = avito['adress'].str.replace('(к)(?=\d)',' корпус ').str.lower()

/tmp/ipykernel_248/2466236294.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  avito['clean_adress'] = avito['adress'].str.replace('(к)(?=\d)',' корпус ').str.lower()


In [34]:
def calc_jaccard(adress_list):
    new_dict = {}
    with db.connect('../data/RE_moscow_db.duckdb') as conn:
        for adress in tqdm(adress_list):
            new_dict[adress] = conn.execute(f'''select Адрес from home_info where Адрес!= 'Адрес' order by jaccard('{adress}',Адрес) desc limit 1''').df().iloc[0,0]
    return new_dict


In [35]:
new_dict = calc_jaccard(avito['clean_adress'].drop_duplicates().tolist())

100%|██████████| 2838/2838 [03:47<00:00, 12.50it/s]


In [36]:
avito['clean_adress'] = avito['clean_adress'].map(new_dict)

In [37]:
dd = avito.merge(df,left_on='clean_adress',right_on='Адрес',how='left' )

In [77]:
dd['n_rooms'] = dd['n_rooms'].str.replace('�','')

In [60]:
dd['Год'] = dd['Год'].where(lambda x: x!='—').astype(float)

In [118]:
plot_df = dd.groupby([dd['Год'].div(5).round().mul(5),'n_rooms','metro_branch'])['rubm2'].mean().reset_index()

In [119]:
import plotly.express as px

In [120]:
px.line(plot_df,x='Год',y='rubm2',color='n_rooms',animation_frame='metro_branch',height=900)